In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import accelerate
import sqlite3
from tqdm.auto import tqdm
import json
import faiss # Faissライブラリ
import shutil # 一時ファイルの削除用

# CUDAのデバッグ用
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# --- 1. GPUの確認 ---
if torch.cuda.is_available():
    print(f"✅ GPU is available. Device: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("⚠️ GPU not found. Running on CPU.")
    device = torch.device("cpu")

✅ GPU is available. Device: NVIDIA RTX A6000


In [2]:
# --- 2. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"
MODEL_CHECKPOINT = "allenai/scibert_scivocab_uncased"

# --- 出力ファイル ---
# (これらは最終的に生成されるファイル)
EMBEDDINGS_OUTPUT_FILE = "data/processed/pretrained_scibert_cls_embeddings.npy"
DOI_MAP_OUTPUT_FILE = "data/processed/pretrained_doi_map.json"
FAISS_INDEX_OUTPUT_FILE = "data/processed/pretrained_scibert.faiss"

# --- 一時ディレクトリ (★再開機能の核) ---
TEMP_EMBED_DIR = "data/processed/embeddings_tmp"
TEMP_DOI_DIR = "data/processed/dois_tmp"

# --- ハイパーパラメータ ---
MAX_LENGTH = 512
DB_READ_BATCH_SIZE = 1000 # DBから一度に読み込む行数
INFERENCE_BATCH_SIZE = 512 # GPUで一度に処理するバッチサイズ

print(f"Configuration set. Inference Batch Size: {INFERENCE_BATCH_SIZE}")

Configuration set. Inference Batch Size: 512


In [3]:
# --- 3. 事前学習済みモデルとトークナイザのロード ---

print(f"Loading tokenizer: {MODEL_CHECKPOINT}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

print(f"Loading PRE-TRAINED model: {MODEL_CHECKPOINT}")
model = AutoModel.from_pretrained(MODEL_CHECKPOINT).to(device)
model.eval() # 評価モードに設定
print("Model and tokenizer loaded successfully.")

Loading tokenizer: allenai/scibert_scivocab_uncased


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Loading PRE-TRAINED model: allenai/scibert_scivocab_uncased


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [4]:
# --- 4. データベースからのデータ読み込み (ジェネレータ) ---

def get_abstract_batches(db_path, batch_size=1000):
    """
    DBからアブストラクトをバッチ単位で読み込むジェネレータ
    """
    print(f"Opening database connection: {db_path}")
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        
        query = "SELECT COUNT(doi) FROM papers WHERE abstract IS NOT NULL AND abstract != ''"
        total_rows = cursor.execute(query).fetchone()[0]
        print(f"Total abstracts to process: {total_rows:,}")
        
        # 最初に総行数を返す
        yield total_rows 
        
        cursor.execute("SELECT doi, abstract FROM papers WHERE abstract IS NOT NULL AND abstract != ''")
        
        batch = []
        for row in cursor: # ここではtqdmを使わない
            batch.append(row)
            if len(batch) >= batch_size:
                yield batch
                batch = []
        if batch:
            yield batch

print("Database generator defined.")

Database generator defined.


In [5]:
# --- 5. 全アブストラクトのベクトル化を実行 ---

# 一時ディレクトリを作成
os.makedirs(TEMP_EMBED_DIR, exist_ok=True)
os.makedirs(TEMP_DOI_DIR, exist_ok=True)

# --- ★再開機能★ ---
# 既に処理済みのバッチインデックスをスキャン
completed_indices = set()
print(f"Scanning {TEMP_DOI_DIR} for completed batches...")
for f in os.listdir(TEMP_DOI_DIR):
    if f.startswith('batch_') and f.endswith('.json'):
        try:
            # batch_00001.json -> 1
            completed_indices.add(int(f[6:11]))
        except:
            pass # ファイル名が不正な場合は無視
print(f"Found {len(completed_indices)} completed batches to skip.")
# --- ここまで ---

# torch.no_grad() で勾配計算を無効化
with torch.no_grad():
    
    batch_generator = get_abstract_batches(DB_PATH, batch_size=DB_READ_BATCH_SIZE)
    total_rows = next(batch_generator)
    total_batches = (total_rows + DB_READ_BATCH_SIZE - 1) // DB_READ_BATCH_SIZE
    
    print("Starting/Resuming embedding generation...")
    
    # tqdmプログレスバーを設定
    pbar = tqdm(enumerate(batch_generator), total=total_batches, desc="Vectorizing Batches")
    
    for i, batch in pbar:
        
        # --- ★再開機能★ ---
        if i in completed_indices:
            pbar.set_description(f"Skipping Batch {i} (already processed)")
            continue # このバッチは処理済みなのでスキップ
        # --- ここまで ---

        pbar.set_description(f"Processing Batch {i}")
        dois, abstracts = zip(*batch)
        
        # バッチ(1000件)をさらに小さなサブバッチ(512件)に分けてGPUで処理
        # (GPUメモリが少ない場合のため)
        batch_embeddings_list = []
        for j in range(0, len(abstracts), INFERENCE_BATCH_SIZE):
            sub_batch_abstracts = abstracts[j : j + INFERENCE_BATCH_SIZE]
            
            inputs = tokenizer(
                list(sub_batch_abstracts), 
                padding="max_length", 
                truncation=True, 
                max_length=MAX_LENGTH, 
                return_tensors="pt"
            ).to(device)
            
            outputs = model(**inputs)
            embeddings = outputs.pooler_output
            batch_embeddings_list.append(embeddings.cpu().numpy())
        
        # サブバッチを結合
        embeddings_cpu = np.vstack(batch_embeddings_list).astype(np.float32)
        
        # --- チャンク（一時ファイル）としてディスクに保存 ---
        embed_filename = os.path.join(TEMP_EMBED_DIR, f"batch_{i:05d}.npy")
        doi_filename = os.path.join(TEMP_DOI_DIR, f"batch_{i:05d}.json")
        
        np.save(embed_filename, embeddings_cpu)
        # ★DOIファイルを最後に保存する（=このバッチの「完了」フラグ）
        with open(doi_filename, 'w') as f:
            json.dump(dois, f)

print(f"\nEmbedding generation complete. All batches saved in chunks.")

Scanning data/processed/dois_tmp for completed batches...
Found 1 completed batches to skip.
Opening database connection: data/processed/s2orc_filtered.db
Total abstracts to process: 11,619,136
Starting/Resuming embedding generation...


Vectorizing Batches:   0%|          | 0/11620 [00:00<?, ?it/s]


Embedding generation complete. All batches saved in chunks.


In [6]:
# --- 6. チャンクの結合と保存 ---
print("Merging embedding chunks from disk...")

# 1. 保存されたDOIチャンクをすべて読み込み、最終的なDOIリストと総数を確定
all_dois = []
doi_files = sorted([f for f in os.listdir(TEMP_DOI_DIR) if f.endswith('.json')])
total_rows_processed = 0

print(f"Reading {len(doi_files)} DOI chunks...")
for f in tqdm(doi_files, desc="Reading DOI chunks"):
    with open(os.path.join(TEMP_DOI_DIR, f), 'r') as fp:
        batch_dois = json.load(fp)
        all_dois.extend(batch_dois)
        total_rows_processed += len(batch_dois)

print(f"Total vectors to merge: {total_rows_processed:,}")

# 2. 最終的な巨大Numpy配列（メモリマップファイル）をディスク上に作成
#    (Shape: [総論文数, 768])
d = 768 # SciBERTの隠れ層サイズ
final_embeddings = np.memmap(
    EMBEDDINGS_OUTPUT_FILE, 
    dtype=np.float32, 
    mode='w+', 
    shape=(total_rows_processed, d)
)

# 3. 一時ファイル（NPY）を順番に読み込み、巨大な配列に書き込む
current_index = 0
print(f"Merging {len(doi_files)} Embedding chunks into {EMBEDDINGS_OUTPUT_FILE}...")
for f in tqdm(doi_files, desc="Merging Embedding Chunks"):
    # batch_00001.json -> batch_00001.npy
    batch_npy_file = os.path.join(TEMP_EMBED_DIR, f.replace('.json', '.npy'))
    
    batch_data = np.load(batch_npy_file)
    
    start_index = current_index
    end_index = start_index + len(batch_data)
    
    final_embeddings[start_index:end_index] = batch_data
    current_index = end_index

# 4. メモリマップを閉じる（ディスクへの書き込みを確定）
final_embeddings.flush()
del final_embeddings # メモリマップファイルへの参照を閉じる
print(f"Final embeddings saved to {EMBEDDINGS_OUTPUT_FILE}")

Merging embedding chunks from disk...
Reading 11620 DOI chunks...


Reading DOI chunks:   0%|          | 0/11620 [00:00<?, ?it/s]

Total vectors to merge: 11,619,136
Merging 11620 Embedding chunks into data/processed/pretrained_scibert_cls_embeddings.npy...


Merging Embedding Chunks:   0%|          | 0/11620 [00:00<?, ?it/s]

Final embeddings saved to data/processed/pretrained_scibert_cls_embeddings.npy


In [7]:
# --- 7. DOIマップの保存と一時ファイルの削除 ---

# 1. DOIマップの保存
print(f"Saving DOI-to-Index map to {DOI_MAP_OUTPUT_FILE}...")
doi_to_index_map = {doi: i for i, doi in enumerate(all_dois)}
with open(DOI_MAP_OUTPUT_FILE, 'w') as f:
    json.dump(doi_to_index_map, f)

# 2. 一時ディレクトリの削除
print(f"Cleaning up temporary directories...")
shutil.rmtree(TEMP_EMBED_DIR)
shutil.rmtree(TEMP_DOI_DIR)

print("\n--- Step 1 (Embedding Generation) Complete ---")
print(f"Total embeddings saved: {len(doi_to_index_map)}")

Saving DOI-to-Index map to data/processed/pretrained_doi_map.json...
Cleaning up temporary directories...

--- Step 1 (Embedding Generation) Complete ---
Total embeddings saved: 11619136


In [9]:
# --- 8. Faissインデックスの構築と保存 ---
print("\n--- Building Faiss Index ---")
d = 768 # SciBERTの次元数

# 1. ファイルサイズから行数（ベクトル数）を計算
file_size = os.path.getsize(EMBEDDINGS_OUTPUT_FILE)
dtype_size = np.dtype(np.float32).itemsize # 4バイト
total_vectors = file_size // (d * dtype_size)

print(f"File size: {file_size / (1024**3):.2f} GB")
print(f"Calculated vector count: {total_vectors:,}")

# 2. np.memmap で読み込みモード ('r') で開く
print(f"Loading embeddings from {EMBEDDINGS_OUTPUT_FILE} (memmap read mode)...")
embeddings_mmap = np.memmap(
    EMBEDDINGS_OUTPUT_FILE,
    dtype=np.float32,
    mode='r',
    shape=(total_vectors, d)
)

# 3. インデックスの構築
# (IndexFlatL2は全データをRAM上のインデックスにコピーするため、十分なRAMが必要です)
index = faiss.IndexFlatL2(d)
print(f"Faiss index type: IndexFlatL2 (Dimensions: {d})")

# ベクトルをインデックスに追加
# (mmapを使うことで、ディスクから少しずつ読み出してRAM上のインデックスにコピーされる)
print("Adding vectors to the index (this may take time)...")
index.add(embeddings_mmap)

print(f"Total vectors in index: {index.ntotal}")

# 4. インデックスをディスクに保存
print(f"Saving Faiss index to {FAISS_INDEX_OUTPUT_FILE}...")
faiss.write_index(index, FAISS_INDEX_OUTPUT_FILE)

print("\n--- Faiss Indexing Complete ---")


--- Building Faiss Index ---
File size: 33.24 GB
Calculated vector count: 11,619,136
Loading embeddings from data/processed/pretrained_scibert_cls_embeddings.npy (memmap read mode)...
Faiss index type: IndexFlatL2 (Dimensions: 768)
Adding vectors to the index (this may take time)...
Total vectors in index: 11619136
Saving Faiss index to data/processed/pretrained_scibert.faiss...

--- Faiss Indexing Complete ---
